In [53]:
# import libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
# %matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_predict, train_test_split

text_mining_keywords = ['cochera', 'sum', 'parrilla', 'pileta']


def vectorize_descripcion(df: pd.DataFrame):
    descripcion = df['descripcion']
    descripcion[descripcion.isna()] = ''

    
    # entrenamos el modelo para buscar las keywords
    vectorizer = CountVectorizer(binary=True)  
    vectorizer.fit(text_mining_keywords)

    # hot encoding de los resultados
    df_countvectorizer = vectorizer.transform(descripcion)

    desc = pd.DataFrame(df_countvectorizer.toarray())
    desc.columns = text_mining_keywords
    
    df.drop(['descripcion'], axis=1, inplace=True)
    return(pd.concat([df, desc], axis=1))
    

def preprocessing(df: pd.DataFrame):
    # guardar los id
    ids = df['id']

    # eliminar columnas innecesarias del modelo
    df.drop(['id', 'pub_inicio', 'pub_fin', 'pub_creada', 'subbarrio', 'titulo'],axis=1, inplace=True)

    df = vectorize_descripcion(df)

    return ids, df


def split_data(df: pd.DataFrame, test_size = 0.05):
    y = df['precio']
    X = df.drop(['precio'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=5)
    return X_train, X_test, y_train, y_test

def get_sample(X: pd.DataFrame, y: pd.DataFrame, sample_size = 0.1):
    size = round(len(X)*sample_size)
    
    idx = np.arange(0, len(X))
    rand_idx = np.random.choice(idx, size, replace=False)
    
    X_sample = X.iloc[rand_idx]
    y_sample = y.iloc[rand_idx]
    return X_sample, y_sample

## Preprocesamiento de los features: imputación, estandarización, one-hot encoding
def make_transformer_pipeline():
    numeric_features = ['lat', 'lon', 'habitaciones', 'dormitorios', 'banios',
       'sup_total', 'sup_cubierta', 'cochera', 'sum', 'parrilla',
       'pileta']
    numeric_transformer = make_pipeline( 
        SimpleImputer(missing_values = np.nan, strategy='mean'),
        StandardScaler(),
    )

    categorical_features = ['barrio']
    categorical_transformer = make_pipeline(
        SimpleImputer(strategy='constant', fill_value='missing'),
        #SimpleImputer(strategy="most_frequent"),
        #KNNImputer(n_neighbors=3, weights="uniform"),     # KNNImputer tarda una eternidad...
        OneHotEncoder(handle_unknown='ignore')
    )

    return make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features)
    )    

## Crear modelo RandomForestRegressor
def create_train_rgr_model(df: pd.DataFrame):
    transformer = make_transformer_pipeline()

    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', TransformedTargetRegressor(RandomForestRegressor(random_state=5), transformer=StandardScaler()))
    ])
    return pipeline

## Crear modelo SVR
def create_train_svr_model(df: pd.DataFrame):
    transformer = make_transformer_pipeline()
    
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('model', TransformedTargetRegressor(SVR(cache_size=1000), transformer=StandardScaler()))
    ])
    return pipeline

## Crear modelo LinearRegression
def create_train_lr_model(df: pd.DataFrame):
    transformer = make_transformer_pipeline()
    lr = LinearRegression()

    pipeline = make_pipeline(transformer, lr)
    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

    return model    

def plot_test_predictions(y_test, y_pred):
    # graficar predicciones contra datos actuales
    plt.figure(figsize = (18,10))
    plot_k = sns.scatterplot(y_test, y_pred)
    plot_k.set(xlabel='Precio test', ylabel='Precio predicción')
    # graficar la identidad
    x_plot = np.linspace(0,np.max(y_test),1000)
    y_plot = x_plot
    plt.plot(x_plot, y_plot, color='r')



In [54]:
df_train = pd.read_excel('propiedades_entrenamiento.xlsx')
df_train.head()

,id,pub_inicio,pub_fin,pub_creada,lat,lon,barrio,subbarrio,habitaciones,dormitorios,banios,sup_total,sup_cubierta,titulo,descripcion,precio
0,1,2019-11-20,9999-12-31,2019-11-20,-34.606891,-58.409890,Almagro,NaN,3.0,NaN,2.0,108.0,108.0,PISO EN VENTA 3 AMB C/ ESC. Y DEP. EN ALMAGRO,Piso en Venta 3 ambientes c/ Escritorio y Depe...,150000
1,3,2019-11-19,2020-02-22,2019-11-19,-34.604912,-58.417065,Almagro,NaN,3.0,1.0,2.0,72.0,63.0,Venta Departamento en Almagro Capital Federal ...,"Departamento en Venta, 3 AMBIENTES + COCHERA E...",144000
2,4,2019-05-21,2019-07-03,2019-05-21,-34.600886,-58.431204,Villa Crespo,NaN,3.0,2.0,1.0,57.0,57.0,DEPARTAMENTO EN VENTA,Xintel (GOP-GOP-8943) Venta de Departamento 3 ...,145000
3,5,2019-05-21,2020-05-22,2019-05-21,-34.628760,-58.453224,Flores,NaN,4.0,3.0,3.0,128.0,117.0,DEPARTAMENTO EN VENTA,Xintel (AGU-AGU-3167) Hermoso 3 ambientes al ...,585000
4,6,2019-07-15,2020-02-13,2019-07-15,-34.572144,-58.480173,Villa Urquiza,NaN,3.0,NaN,2.0,NaN,60.0,Departamento en Venta ubicado en Villa Urquiza R,"Venta - Departamento - Argentina, Buenos Aires...",237000


In [55]:
ids, df_train = preprocessing(df_train)

In [56]:
X_train, X_test, y_train, y_test = split_data(df_train)

## Búsqueda de hiperparámetros óptimos de SVR

In [19]:
# Tomamos muestra para el grid search (búsqueda de hiperparámetros óptimos)
X_train_sample, y_train_sample = get_sample(X_train, y_train, sample_size=0.2)
X_train_sample.shape, y_train_sample.shape

In [23]:
# Entrenar modelo SVR con búsqueda de hiperparámetros
tuned_parameters = [{'model__regressor__kernel': ['rbf'], 
                    'model__regressor__gamma': np.logspace(-3, 3, 3),
                    'model__regressor__C': np.logspace(-3, 3, 4)},
                    # {'model__regressor__kernel': ['linear'], 
                    #  'model__regressor__C': [1, 10, 100, 1000]}
                    ]
model_svr = create_train_svr_model(df_train)

model = GridSearchCV(model_svr, tuned_parameters, n_jobs=2, scoring=('r2', 'neg_root_mean_squared_error'), refit='neg_root_mean_squared_error')
model.fit(X_train_sample, y_train_sample)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                   

In [25]:
print("Mejores parámetros encontrados:")
print()
print(model.best_params_)
print()
print("Grid scores:")
df_scores=pd.DataFrame(data=model.cv_results_)
df_scores

Mejores parámetros encontrados:

{'model__regressor__C': 1000.0, 'model__regressor__gamma': 0.001, 'model__regressor__kernel': 'rbf'}

Grid scores:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__regressor__C,param_model__regressor__gamma,param_model__regressor__kernel,params,split0_test_r2,split1_test_r2,...,std_test_r2,rank_test_r2,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error
0,2.918163,0.219134,0.483093,0.009782,0.001,0.001,rbf,"{'model__regressor__C': 0.001, 'model__regress...",-0.047424,-0.044655,...,0.003736,11,-281648.894792,-277940.101335,-316909.833328,-258192.051743,-297849.619197,-286508.100079,19757.107644,11
1,2.493811,0.041938,0.487898,0.016805,0.001,1,rbf,"{'model__regressor__C': 0.001, 'model__regress...",-0.044663,-0.043662,...,0.003118,10,-281277.370243,-277807.982556,-316441.886967,-257642.615543,-297774.914699,-286188.954002,19792.585556,10
2,2.067730,0.129361,0.438728,0.019836,0.001,1000,rbf,"{'model__regressor__C': 0.001, 'model__regress...",-0.075188,-0.073905,...,0.006978,12,-285357.252014,-281804.331787,-319875.645458,-261998.257414,-301423.653404,-290091.828015,19472.203578,12
3,2.212726,0.068540,0.407634,0.029148,0.1,0.001,rbf,"{'model__regressor__C': 0.1, 'model__regressor...",0.323229,0.347595,...,0.031280,5,-226395.376346,-219646.099168,-266709.106795,-204226.632240,-243889.004756,-232173.243861,21453.722708,5
4,2.402511,0.021212,0.415643,0.049212,0.1,1,rbf,"{'model__regressor__C': 0.1, 'model__regressor...",0.202753,0.196523,...,0.027862,6,-245721.586878,-243753.835462,-285686.716964,-220666.555122,-266059.256973,-252377.590280,22004.305048,6
5,2.397013,0.124993,0.452278,0.012893,0.1,1000,rbf,"{'model__regressor__C': 0.1, 'model__regressor...",-0.024252,-0.023397,...,0.002624,9,-278516.024146,-275097.647954,-314125.386174,-254533.638653,-295533.696482,-283561.278682,20085.790892,9
6,2.075249,0.124180,0.363523,0.014269,10,0.001,rbf,"{'model__regressor__C': 10.0, 'model__regresso...",0.590947,0.529831,...,0.042437,2,-176009.600726,-186462.568534,-227208.342962,-180060.470869,-203095.108546,-194567.218327,18754.648331,2
7,3.220416,0.132777,0.371372,0.015516,10,1,rbf,"{'model__regressor__C': 10.0, 'model__regresso...",0.540016,0.484272,...,0.064944,3,-186645.706584,-195287.803646,-246989.997309,-176764.297572,-225735.499566,-206284.660935,26127.880833,3
8,3.931039,0.114186,0.561106,0.066960,10,1000,rbf,"{'model__regressor__C': 10.0, 'model__regresso...",0.150526,0.151786,...,0.028270,8,-253642.297783,-250448.025874,-295240.303192,-234635.219772,-279938.569648,-262780.883254,21798.126679,8
9,9.229634,0.379213,0.317460,0.010424,1000,0.001,rbf,"{'model__regressor__C': 1000.0, 'model__regres...",0.685938,0.744384,...,0.049947,1,-154224.979550,-137486.152583,-188998.745645,-141567.026843,-184847.417176,-161424.864359,21578.548993,1


## Búsqueda de hiperparámetros RandomForestRegressor

In [60]:
# Tomamos muestra para el grid search (búsqueda de hiperparámetros óptimos)
X_train_sample, y_train_sample = get_sample(X_train, y_train, sample_size=0.2)
X_train_sample.shape, y_train_sample.shape

((9642, 12), (9642,))

In [61]:
# Entrenar modelo RandomForestRegressor con búsqueda de hiperparámetros
tuned_parameters = {'model__regressor__n_estimators': [100, 200, 400], 
                    'model__regressor__max_depth': [30, 60, 100, 300],
                    'model__regressor__max_features': [2, 3],
                    'model__regressor__min_samples_leaf': [3, 4, 5],
                    'model__regressor__min_samples_split': [8, 10, 12]
                    }
model_rgr = create_train_rgr_model(df_train)

model = GridSearchCV(model_rgr, tuned_parameters, n_jobs=2, scoring='neg_root_mean_squared_error', refit='neg_root_mean_squared_error', verbose=5)
model.fit(X_train_sample, y_train_sample)

KeyboardInterrupt: 

In [27]:
df_test = pd.read_excel('https://github.com/arielfaur/prediccion-precio-propiedades-fmap-2020/raw/master/propiedades_prueba.xlsx')
df_test.head()

,id,pub_inicio,pub_fin,pub_creada,lat,lon,barrio,subbarrio,habitaciones,dormitorios,banios,sup_total,sup_cubierta,titulo,descripcion,precio
0,2,2019-10-23,9999-12-31,2019-10-23,-34.586330,-58.508957,Villa Devoto,NaN,1.0,NaN,NaN,45,37,Ladines 3200 pb - U$D 124.000 - Departamento ...,Impecable 2 ambientes con patio cubierto con t...,NaN
1,34,2019-05-07,2019-06-23,2019-05-07,-34.581699,-58.433547,Palermo,Palermo Hollywood,1.0,1.0,1.0,53,40,VENTA MONOAMBIENTE ESPECTACULAR EN COLEGIALES,Monoambiente impecable divisible a dos ambient...,NaN
2,54,2019-10-13,2019-12-23,2019-10-13,-34.628187,-58.429586,Caballito,NaN,2.0,1.0,1.0,42,38,2 ambientes divisibles a estrenar en Palmera C...,Palmera Caballito: Torre con amenities de gran...,NaN
3,101,2019-08-25,2019-09-14,2019-08-25,-34.593716,-58.440695,Palermo,NaN,2.0,1.0,1.0,62,38,Depto 2 amb c/terraza prpia Palermo Queens out...,Corredor Responsable: Ariel Champanier - CUCIC...,NaN
4,103,2019-10-23,2019-11-09,2019-10-23,-34.591336,-58.447218,Villa Crespo,NaN,2.0,1.0,1.0,44,40,Departamento de dos ambientes con cochera en v...,Departamento de 2 ambientes con cochera ubicad...,NaN


In [28]:
ids, df_test = preprocessing(df_test)

In [43]:
model_svr.set_params(model__regressor__C=1000, model__regressor__gamma=0.001)

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                 

In [49]:
model_svr.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                 

In [50]:
predicted = model_svr.predict(df_test)
data = pd.DataFrame(data={'id' : ids.values, 'precio': np.round(predicted).astype(int) })

In [51]:
data


,id,precio
0,2,129019
1,34,145342
2,54,122122
3,101,116858
4,103,133762
...,...,...
2495,53190,118715
2496,53221,223096
2497,53233,114175
2498,53243,144380


In [52]:
#from google.colab import files
data.to_csv('propiedades_prediccion.csv', index=False)
#files.download('propiedades_prediccion.csv')